In [16]:
from astropy.table import Table, vstack, unique, SortedArray
import glob
import numpy
import time
from datetime import date, timedelta, datetime
import psycopg2
import sqlite3
import pandas as pd
from functools import reduce
from desispec.io import read_spectra, write_spectra
from desispec.spectra import Spectra
from desispec.coaddition import coadd
#from coaddFibermapNoExpid import coadd_fibermap_no_expid
from subclassSpectraNoExpid import spectra_no_expid

from coaddNoExpid import coadd_no_expid
#from updateNoExpid import update_no_expid

import matplotlib.pyplot as plt
import os
from desiutil.log import get_logger, DEBUG

In [ ]:
#get array of yyyymmdds to loop through
#getUnprocessedDates.py

In [4]:
f = open('/global/cscratch1/sd/clepart/desi_pg.txt')
file = f.read()
db_name, db_user, db_pwd, db_host = file.split()
conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_pwd, host=db_host)
    
# Open a cursor to perform database operations
cur = conn.cursor()
cur.execute("""SELECT DISTINCT yyyymmdd from fibermap_daily WHERE yyyymmdd > 20210604""") #most recent, remove in future
desi_arr = cur.fetchall()

cur.close()
conn.close()

In [5]:
#for i in range(len(desi_arr)):
#    print(desi_arr[i])
print(len(desi_arr))
print(type(desi_arr))

27
<class 'list'>


In [6]:
filename_conn = "/global/cfs/cdirs/desi/science/td/daily-search/transients_search.db"
conn = sqlite3.connect(filename_conn)

trans_arr = conn.execute("""SELECT DISTINCT yyyymmdd from unprocessed_exposures""").fetchall()

conn.close()

In [7]:
#for i in range(len(trans_arr)):
#    print(trans_arr[i])
print(len(trans_arr))

7


In [8]:
night_arr = numpy.setdiff1d(desi_arr, trans_arr)

In [9]:
for i in range(10):
    print(night_arr[i])
print('len(night_arr): ' + str(len(night_arr)))

20210606
20210607
20210614
20210615
20210616
20210617
20210618
20210619
20210620
20210621
len(night_arr): 20


In [10]:
#for a in query_arr:
start_time = time.time()
#get time duration stamp on process that reads in all spectra files for one night
###for a in night_arr: #first loop
    #yyyymmd = a
yyyymmdd = 20210406
# read in and store in one place all the fibermap information in the spectra files
dats=[]
for filename in glob.glob(f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/*/{yyyymmdd}/spectra-*.fits"):
    t = Table.read(filename, format='fits',hdu=1, memmap=True)[['TARGETID','TARGET_RA','TARGET_DEC','TILEID','OBJTYPE','PETAL_LOC','NIGHT','MJD']]
    t=t[t['OBJTYPE']=='TGT']
    dats.append(t)
dats=vstack(dats, join_type='inner',metadata_conflicts='silent')
# group all the observations by TARGET_RA and TARGET_DEC
# note that this is more reliable than grouping by TARGETID as TARGETID is NOT a unique identifier of RA and DEC
dats_group = dats.group_by(['TARGET_RA','TARGET_DEC'])
print("--- read in file:  %s seconds ---" % (time.time() - start_time))
#get RA_DEC, NIGHT for multiple pairs of tile/petal

--- read in file:  90.52970814704895 seconds ---


In [10]:
print(len(dats_group)) #ungrouped
for i in range(5):
    print(dats_group[i])
print(len(dats_group.groups)) #grouped 
for g in range(5):
    print(dats_group.groups[g])

284964
     TARGETID         TARGET_RA         TARGET_DEC    TILEID OBJTYPE PETAL_LOC  NIGHT        MJD      
----------------- ------------------ ---------------- ------ ------- --------- -------- --------------
39627841574276999 148.43269333549472 2.29989340162531      1     TGT         3 20210406 59311.13373059
     TARGETID         TARGET_RA         TARGET_DEC    TILEID OBJTYPE PETAL_LOC  NIGHT        MJD      
----------------- ------------------ ---------------- ------ ------- --------- -------- --------------
39627841574276999 148.43269333549472 2.29989340162531      1     TGT         3 20210406 59311.14474809
     TARGETID         TARGET_RA         TARGET_DEC    TILEID OBJTYPE PETAL_LOC  NIGHT        MJD      
----------------- ------------------ ---------------- ------ ------- --------- -------- --------------
39627841574276999 148.43269333549472 2.29989340162531      1     TGT         3 20210406 59311.15405799
     TARGETID         TARGET_RA         TARGET_DEC     TILEID OBJT

In [11]:

groups01 = dats_group.groups[0:2]
groups01 = groups01.group_by(['TARGET_RA','TARGET_DEC'])

#for g in groups01.groups:
for g in dats_group.groups:
    #print(type(g)) <class 'astropy.table.table.Table'>
    start_time = time.time()
    d = g.group_by(['NIGHT'])
    du = unique(d, keys=['NIGHT']) 
    print("du[0]" + str(du[0]))
    s0 = Spectra() #spectra_no_expid() ### all spectra
    s0c = Spectra() #spectra_no_expid() ### only current spectra
    s0r = Spectra() #spectra_no_expid() ### only reference spectra
    s0d = Spectra() #differenced spectra

    f0 = f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/{du[0]['TILEID']}/{yyyymmdd}/spectra-{du[0]['PETAL_LOC']}-{du[0]['TILEID']}-thru{yyyymmdd}.fits"
    #this is being done multiple times: need to find way to implement select(targets = ['TID']) for files already read in
    s0 = read_spectra(f0)
    s0c = s0.select(nights = [yyyymmdd])
    coadd_no_expid(s0c)
    for t in du:
        
        if (t['NIGHT'] == (yyyymmdd-1)):
            s0c.update(s0.select(nights = [yyyymmdd-1]))
            print("updated %s" % s0.select(nights = [yyyymmdd-1]))
            coadd_no_expid(s0c)
        elif t['NIGHT'] < yyyymmdd-1: # in future, use getPrevObsdate for days > 30 days ago
            if (s0r.num_spectra() == 0): 
            # first (or only) ref night found
                s0r = s0.select([t['NIGHT']])
                print(t['NIGHT'])
            else:
                s0r = s0.update(s0.select(nights = [t['NIGHT']]))
                print(t['NIGHT'])
            coadd_no_expid(s0r)
            for i in s0r.flux:
            #flip signs of r0a.flux
                s0r.flux[i] = -1*r0a.flux[i]
    
    if s0r.num_spectra() != 0:
    # if there is something to difference
        s0d.update(s0c) #not working, produce NoneType
        s0d.update(s0r) #not working, produces NoneType
        print("s0d differenced")
        coadd_no_expid(s0d)   #how to store differenced spectra as one per reference spectra will be produced
    else:
        print("no reference nights")
    print("--- group_by NIGHT  %s seconds ---" % (time.time() - start_time))
                


du[0]     TARGETID         TARGET_RA         TARGET_DEC    TILEID OBJTYPE PETAL_LOC  NIGHT        MJD      
----------------- ------------------ ---------------- ------ ------- --------- -------- --------------
39627841574276999 148.43269333549472 2.29989340162531      1     TGT         3 20210406 59311.13373059
INFO:spectra.py:253:read_spectra: iotime 2.151 sec to read spectra-3-1-thru20210406.fits at 2021-07-19T11:39:18.217306
no reference nights
--- group_by NIGHT  6.413954496383667 seconds ---
du[0]     TARGETID         TARGET_RA         TARGET_DEC     TILEID OBJTYPE PETAL_LOC  NIGHT        MJD      
----------------- ----------------- ------------------ ------ ------- --------- -------- --------------
39627841574277409 148.4503883832226 2.2991424581727253     12     TGT         3 20210405 59310.11613136
INFO:spectra.py:253:read_spectra: iotime 1.434 sec to read spectra-3-12-thru20210406.fits at 2021-07-19T11:39:24.222593


/global/common/software/desi/cori/desiconda/20200801-1.4.0-spec/conda/lib/python3.8/site-packages/astropy/table/table.py:1679: StringTruncateWarning: truncated right side string(s) longer than 4 character(s) during assignment
  col[item] = val


ValueError: could not convert string to float: b'PSF'

In [22]:
groups01 = dats_group.groups[0:2]
groups01 = groups01.group_by(['TARGET_RA','TARGET_DEC'])

#for g in groups01.groups:
for g in dats_group.groups:
    #print(type(g)) <class 'astropy.table.table.Table'>
    start_time = time.time()
    d = g.group_by(['NIGHT'])
    du = unique(d, keys=['NIGHT']) 
    print("du[0]" + str(du[0]))
    s0 = spectra_no_expid() ### all spectra
    s0c = spectra_no_expid() ### only current spectra
    s0r = spectra_no_expid() ### only reference spectra
    s0d = spectra_no_expid() ### differenced spectra

    f0 = f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/{du[0]['TILEID']}/{yyyymmdd}/spectra-{du[0]['PETAL_LOC']}-{du[0]['TILEID']}-thru{yyyymmdd}.fits"
    #this is being done multiple times: need to find way to implement select(targets = ['TID']) for files already read in
    s0 = read_spectra(f0)
    s0c = s0.select(nights = [yyyymmdd])
    print('after read_spectra ' + str(type(s0)))
    print('after select ' + str(type(s0c)))
    coadd_no_expid(s0c)
    for t in du:
        
        if (t['NIGHT'] == (yyyymmdd-1)):
            s0c.update(s0.select(nights = [yyyymmdd-1]))
            print("updated %s" % s0.select(nights = [yyyymmdd-1]))
            coadd_no_expid(s0c)
        elif t['NIGHT'] < yyyymmdd-1: # in future, use getPrevObsdate for days > 30 days ago
            if (s0r.num_spectra() == 0): 
            # first (or only) ref night found
                s0r = s0.select([t['NIGHT']])
                print(t['NIGHT'])
            else:
                s0r = s0.update(s0.select(nights = [t['NIGHT']]))
                print(t['NIGHT'])
            coadd_no_expid(s0r)
            for i in s0r.flux:
            #flip signs of r0a.flux
                s0r.flux[i] = -1*r0a.flux[i]
    
    if s0r.num_spectra() != 0:
    # if there is something to difference
        s0d.update(s0c) #not working 
        s0d.update(s0r) #not working
        print("s0d differenced")
        coadd_no_expid(s0d)   #how to store differenced spectra as one per reference spectra will be produced
    else:
        print("no reference nights")
    print("--- group_by NIGHT  %s seconds ---" % (time.time() - start_time))
                


du[0]     TARGETID         TARGET_RA         TARGET_DEC    TILEID OBJTYPE PETAL_LOC  NIGHT        MJD      
----------------- ------------------ ---------------- ------ ------- --------- -------- --------------
39627841574276999 148.43269333549472 2.29989340162531      1     TGT         3 20210406 59311.13373059
INFO:spectra.py:253:read_spectra: iotime 1.796 sec to read spectra-3-1-thru20210406.fits at 2021-07-20T11:32:51.087513
after read_spectra <class 'desispec.spectra.Spectra'>
after select <class 'desispec.spectra.Spectra'>
no reference nights
--- group_by NIGHT  6.1589484214782715 seconds ---
du[0]     TARGETID         TARGET_RA         TARGET_DEC     TILEID OBJTYPE PETAL_LOC  NIGHT        MJD      
----------------- ----------------- ------------------ ------ ------- --------- -------- --------------
39627841574277409 148.4503883832226 2.2991424581727253     12     TGT         3 20210405 59310.11613136
INFO:spectra.py:253:read_spectra: iotime 1.385 sec to read spectra-3-12-thru2

/global/common/software/desi/cori/desiconda/20200801-1.4.0-spec/conda/lib/python3.8/site-packages/astropy/table/table.py:1679: StringTruncateWarning: truncated right side string(s) longer than 4 character(s) during assignment
  col[item] = val


ValueError: could not convert string to float: b'PSF'

In [6]:
"""   TARGETID         TARGET_RA      ...  NIGHT        MJD      
----------------- ------------------ ... -------- --------------
39627841574276999 148.43269333549472 ... 20210406 59311.13373059
39627841574276999 148.43269333549472 ... 20210406 59311.14474809
39627841574276999 148.43269333549472 ... 20210406 59311.15405799
     TARGETID         TARGET_RA     ...  NIGHT        MJD      
----------------- ----------------- ... -------- --------------
39627841574277409 148.4503883832226 ... 20210405 59310.11613136
39627841574277409 148.4503883832226 ... 20210406 59311.11739078
     TARGETID         TARGET_RA      ...  NIGHT        MJD      
----------------- ------------------ ... -------- --------------
39627847614075911 148.45238860222133 ... 20210406 59311.13373059
39627847614075911 148.45238860222133 ... 20210406 59311.14474809
39627847614075911 148.45238860222133 ... 20210406 59311.15405799
     TARGETID         TARGET_RA      ...  NIGHT        MJD      
----------------- ------------------ ... -------- --------------
39627847614075954 148.45408201409734 ... 20210406 59311.13373059
39627847614075954 148.45408201409734 ... 20210406 59311.14474809
39627847614075954 148.45408201409734 ... 20210406 59311.15405799
     TARGETID         TARGET_RA     ...  NIGHT        MJD      
----------------- ----------------- ... -------- --------------
39627835534479542 148.4658610579723 ... 20210406  59311.1337306
39627835534479542 148.4658610579723 ... 20210406 59311.14474806
39627835534479542 148.4658610579723 ... 20210406 59311.15405797
for g in dats_group.groups:
    print(g)
"""

'   TARGETID         TARGET_RA      ...  NIGHT        MJD      \n----------------- ------------------ ... -------- --------------\n39627841574276999 148.43269333549472 ... 20210406 59311.13373059\n39627841574276999 148.43269333549472 ... 20210406 59311.14474809\n39627841574276999 148.43269333549472 ... 20210406 59311.15405799\n     TARGETID         TARGET_RA     ...  NIGHT        MJD      \n----------------- ----------------- ... -------- --------------\n39627841574277409 148.4503883832226 ... 20210405 59310.11613136\n39627841574277409 148.4503883832226 ... 20210406 59311.11739078\n     TARGETID         TARGET_RA      ...  NIGHT        MJD      \n----------------- ------------------ ... -------- --------------\n39627847614075911 148.45238860222133 ... 20210406 59311.13373059\n39627847614075911 148.45238860222133 ... 20210406 59311.14474809\n39627847614075911 148.45238860222133 ... 20210406 59311.15405799\n     TARGETID         TARGET_RA      ...  NIGHT        MJD      \n---------------

In [11]:
dats_night = dats.group_by(['NIGHT'])
"""for g in dats_night.groups:
    print(g)
     TARGETID         TARGET_RA      ...  NIGHT        MJD      
----------------- ------------------ ... -------- --------------
39633353032337659 259.43455471094023 ... 20210217 59263.50963418
39633353036529697  259.6437859697935 ... 20210217 59263.50963418
39633353036531389  259.8235175322709 ... 20210217 59263.50963418
39633356308088421 260.22311821469606 ... 20210217 59263.50963418
39633356303893786 259.71574742626615 ... 20210217 59263.50963418
39633353036532442  259.9200173727265 ... 20210217 59263.50963418
39633356303896334  259.9490785198656 ... 20210217 59263.50963418
39633353036529984  259.6720514324222 ... 20210217 59263.50963418
39633356303896872  259.9978988150692 ... 20210217 59263.50963418
39633356303893225 259.66050986580046 ... 20210217 59263.50963418
              ...                ... ...      ...            ...
39633381629103056 259.80583808199555 ... 20210217 59263.52120874
39633378542095409 259.66379160329393 ... 20210217 59263.52120874
39633384703527094  260.6105835140099 ... 20210217 59263.52120874
39633381629101801  259.6603131652032 ... 20210217 59263.52120874
39633384703527900 260.69292818061285 ... 20210217 59263.52120874
39633381629103622   259.874333485421 ... 20210217 59263.52120874
39633384703527513  260.6535535806872 ... 20210217 59263.52120874
39633381637491532 260.75617115437495 ... 20210217 59263.52120874
39633384703529066   260.804403644741 ... 20210217 59263.52120874
39633378542095156 259.62955648091435 ... 20210217 59263.52120874
39633384703528656 260.76377328910115 ... 20210217 59263.52120874
Length = 11070 rows
"""

'for g in dats_night.groups:\n    print(g)\n     TARGETID         TARGET_RA      ...  NIGHT        MJD      \n----------------- ------------------ ... -------- --------------\n39633353032337659 259.43455471094023 ... 20210217 59263.50963418\n39633353036529697  259.6437859697935 ... 20210217 59263.50963418\n39633353036531389  259.8235175322709 ... 20210217 59263.50963418\n39633356308088421 260.22311821469606 ... 20210217 59263.50963418\n39633356303893786 259.71574742626615 ... 20210217 59263.50963418\n39633353036532442  259.9200173727265 ... 20210217 59263.50963418\n39633356303896334  259.9490785198656 ... 20210217 59263.50963418\n39633353036529984  259.6720514324222 ... 20210217 59263.50963418\n39633356303896872  259.9978988150692 ... 20210217 59263.50963418\n39633356303893225 259.66050986580046 ... 20210217 59263.50963418\n              ...                ... ...      ...            ...\n39633381629103056 259.80583808199555 ... 20210217 59263.52120874\n39633378542095409 259.6637916032

In [18]:
#dats_grp_night = dats_group.group_by(['NIGHT'])

for g in dats_grp_night.groups:
    print(g)
    

     TARGETID         TARGET_RA      ...  NIGHT        MJD      
----------------- ------------------ ... -------- --------------
39633365946598254  257.1130151968086 ... 20210217  59263.5096341
39633365946598254  257.1130151968086 ... 20210217 59263.51532577
39633365946598254  257.1130151968086 ... 20210217 59263.52120868
39633365946598327 257.12237460175396 ... 20210217  59263.5096341
39633365946598327 257.12237460175396 ... 20210217 59263.51532577
39633365946598327 257.12237460175396 ... 20210217 59263.52120868
39633362746344215 257.16189769657746 ... 20210217  59263.5096341
39633362746344215 257.16189769657746 ... 20210217 59263.51532577
39633362746344215 257.16189769657746 ... 20210217 59263.52120868
39633362746344248  257.1661909403572 ... 20210217  59263.5096341
              ...                ... ...      ...            ...
39633365996932481 263.07301657145723 ... 20210217 59263.51532587
39633365996932481 263.07301657145723 ... 20210217 59263.52120878
39633365996932558  263.08

In [ ]:
dats_tile_petal = dats_group.group_by(['TILEID', 'PETAL_LOC'])
#for d in dats_tile_petal.groups:
#    for i in d:
#        print(i)

In [17]:
dats_grp_night = dats_group.group_by(['NIGHT'])
"""
for g in dats_grp_night.groups:
    print(g)
    
     TARGETID         TARGET_RA      ...  NIGHT        MJD      
----------------- ------------------ ... -------- --------------
39633365946598254  257.1130151968086 ... 20210217  59263.5096341
39633365946598254  257.1130151968086 ... 20210217 59263.51532577
39633365946598254  257.1130151968086 ... 20210217 59263.52120868
39633365946598327 257.12237460175396 ... 20210217  59263.5096341
39633365946598327 257.12237460175396 ... 20210217 59263.51532577
39633365946598327 257.12237460175396 ... 20210217 59263.52120868
39633362746344215 257.16189769657746 ... 20210217  59263.5096341
39633362746344215 257.16189769657746 ... 20210217 59263.51532577
39633362746344215 257.16189769657746 ... 20210217 59263.52120868
39633362746344248  257.1661909403572 ... 20210217  59263.5096341
              ...                ... ...      ...            ...
39633365996932481 263.07301657145723 ... 20210217 59263.51532587
39633365996932481 263.07301657145723 ... 20210217 59263.52120878
39633365996932558  263.0805793297551 ... 20210217  59263.5096342
39633365996932558  263.0805793297551 ... 20210217 59263.51532587
39633365996932558  263.0805793297551 ... 20210217 59263.52120878
39633365996932762  263.1016727163152 ... 20210217  59263.5096342
39633365996932762  263.1016727163152 ... 20210217 59263.51532587
39633365996932762  263.1016727163152 ... 20210217 59263.52120878
39633365996932944 263.12149276925805 ... 20210217  59263.5096342
39633365996932944 263.12149276925805 ... 20210217 59263.51532587
39633365996932944 263.12149276925805 ... 20210217 59263.52120878
Length = 11070 rows
"""

'\nfor g in dats_grp_night.groups:\n    print(g)\n    \n     TARGETID         TARGET_RA      ...  NIGHT        MJD      \n----------------- ------------------ ... -------- --------------\n39633365946598254  257.1130151968086 ... 20210217  59263.5096341\n39633365946598254  257.1130151968086 ... 20210217 59263.51532577\n39633365946598254  257.1130151968086 ... 20210217 59263.52120868\n39633365946598327 257.12237460175396 ... 20210217  59263.5096341\n39633365946598327 257.12237460175396 ... 20210217 59263.51532577\n39633365946598327 257.12237460175396 ... 20210217 59263.52120868\n39633362746344215 257.16189769657746 ... 20210217  59263.5096341\n39633362746344215 257.16189769657746 ... 20210217 59263.51532577\n39633362746344215 257.16189769657746 ... 20210217 59263.52120868\n39633362746344248  257.1661909403572 ... 20210217  59263.5096341\n              ...                ... ...      ...            ...\n39633365996932481 263.07301657145723 ... 20210217 59263.51532587\n39633365996932481 26

In [6]:
"""     TARGETID         TARGET_RA          TARGET_DEC     TILEID OBJTYPE PETAL_LOC  NIGHT        MJD      
----------------- ------------------ ------------------ ------ ------- --------- -------- --------------
39633353032337659 259.43455471094023 57.283373743175126  80738     TGT         0 20210217 59263.50963418
     TARGETID         TARGET_RA         TARGET_DEC    TILEID OBJTYPE PETAL_LOC  NIGHT        MJD      
----------------- ----------------- ----------------- ------ ------- --------- -------- --------------
39633353036529697 259.6437859697935 57.32376101800808  80738     TGT         0 20210217 59263.50963418
     TARGETID         TARGET_RA        TARGET_DEC    TILEID OBJTYPE PETAL_LOC  NIGHT        MJD      
----------------- ----------------- ---------------- ------ ------- --------- -------- --------------
39633353036531389 259.8235175322709 57.3445196407634  80738     TGT         0 20210217 59263.50963418
     TARGETID         TARGET_RA          TARGET_DEC    TILEID OBJTYPE PETAL_LOC  NIGHT        MJD      
----------------- ------------------ ----------------- ------ ------- --------- -------- --------------
39633356308088421 260.22311821469606 57.38917709488297  80738     TGT         0 20210217 59263.50963418
     TARGETID         TARGET_RA          TARGET_DEC    TILEID OBJTYPE PETAL_LOC  NIGHT        MJD      
----------------- ------------------ ----------------- ------ ------- --------- -------- --------------
39633356303893786 259.71574742626615 57.45648341909634  80738     TGT         0 20210217 59263.50963418
     TARGETID         TARGET_RA         TARGET_DEC    TILEID OBJTYPE PETAL_LOC  NIGHT        MJD      
----------------- ----------------- ----------------- ------ ------- --------- -------- --------------
39633353036532442 259.9200173727265 57.36314750292774  80738     TGT         0 20210217 59263.50963418
     TARGETID         TARGET_RA         TARGET_DEC     TILEID OBJTYPE PETAL_LOC  NIGHT        MJD      
----------------- ----------------- ------------------ ------ ------- --------- -------- --------------
39633356303896334 259.9490785198656 57.517448371318316  80738     TGT         0 20210217 59263.50963418
     TARGETID         TARGET_RA         TARGET_DEC    TILEID OBJTYPE PETAL_LOC  NIGHT        MJD      
----------------- ----------------- ----------------- ------ ------- --------- -------- --------------
39633353036529984 259.6720514324222 57.26902542404255  80738     TGT         0 20210217 59263.50963418
     TARGETID         TARGET_RA         TARGET_DEC    TILEID OBJTYPE PETAL_LOC  NIGHT        MJD      
----------------- ----------------- ----------------- ------ ------- --------- -------- --------------
39633356303896872 259.9978988150692 57.37950468844931  80738     TGT         0 20210217 59263.50963418
dats_night = dats.group_by(['NIGHT'])
"""

"     TARGETID         TARGET_RA          TARGET_DEC     TILEID OBJTYPE PETAL_LOC  NIGHT        MJD      \n----------------- ------------------ ------------------ ------ ------- --------- -------- --------------\n39633353032337659 259.43455471094023 57.283373743175126  80738     TGT         0 20210217 59263.50963418\n     TARGETID         TARGET_RA         TARGET_DEC    TILEID OBJTYPE PETAL_LOC  NIGHT        MJD      \n----------------- ----------------- ----------------- ------ ------- --------- -------- --------------\n39633353036529697 259.6437859697935 57.32376101800808  80738     TGT         0 20210217 59263.50963418\n     TARGETID         TARGET_RA        TARGET_DEC    TILEID OBJTYPE PETAL_LOC  NIGHT        MJD      \n----------------- ----------------- ---------------- ------ ------- --------- -------- --------------\n39633353036531389 259.8235175322709 57.3445196407634  80738     TGT         0 20210217 59263.50963418\n     TARGETID         TARGET_RA          TARGET_DEC    TILEI

In [ ]:
dats_night = 

In [ ]:
start_time_i = time.time()
for i in len(ans): #103753
    for j in len(ans[i]): #e.g. 3
        start_time_j = time.time()
        s0a = Spectra()
        print(s0a.num_spectra())
        while s0a.num_spectra()==0:
            for j in range(len(ans[i])):
                if s0a.num_spectra()!=0: #this means a current spectra has been found and stored as s0a
                    if ans[i][j][2]==yyyymmdd:
                        f0b=f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/{ans[i][j][3]}/{ans[i][j][2]}/spectra-{ans[i][j][4]}-{ans[i][j][3]}-thru{ans[i][j][2]}.fits"
                        s0b=read_spectra(f0b)
                        print(s0b.num_spectra()) #1500
                        coadd(s0b)
                        print(s0b.num_spectra()) #500  
                        s0a.update_no_expid(s0b) #not working, produces NoneType
                        print(s0a.num_spectra()) #need to coadd?
                    elif ans[i][j][2] < yyyymmdd:
                        #makes reference spectra,r0a
                        f0a=f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/{ans[i][j][3]}/{ans[i][j][2]}/spectra-{ans[i][j][4]}-{ans[i][j][3]}-thru{ans[i][j][2]}.fits"
                        r0a=read_spectra(f0a)
                        print(r0a.num_spectra()) #1500
                        coadd(r0a)
                        print(r0a.num_spectra()) #500
                        #flip signs of r0a.flux
                        for i in r0a.flux:
                            r0a.flux[i] = -1*r0a.flux[i]
                        #flip signs of r0a.wave
                        for i in r0a.wave:
                            r0a.wave[i] = -1*r0a.wave[i]
                        d0a = Spectra()
                    d0a = Spectra()
                    d0a.update_no_expid(s0a)#not working, produce NoneType
                    d0a.update_no_expid(r0a)#not working, produces NoneType
                    coadd(d0a) #how to store differenced spectra as one per reference spectra will be produced
                else if:
                    print(ans[i][j][2])
                    if ans[i][j][2]==yyyymmdd: #this makes a current spectra
                        f0a=f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/{ans[i][j][3]}/{ans[i][j][2]}/spectra-{ans[i][j][4]}-{ans[4][j][3]}-thru{ans[i][j][2]}.fits"
                        s0a=read_spectra(f0a)
                        print(s0a.num_spectra()) #1500
                        coadd(s0a)
                        print(s0a.num_spectra()) #500  
                    elif ans[i][j][2] < yyyymmdd:
                        #makes reference spectra,r0a
                        f0a=f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/{ans[i][j][3]}/{ans[i][j][2]}/spectra-{ans[i][j][4]}-{ans[i][j][3]}-thru{ans[i][j][2]}.fits"
                        r0a=read_spectra(f0a)
                        print(r0a.num_spectra()) #1500
                        coadd(r0a)
                        print(r0a.num_spectra()) #500
                        #flip signs of r0a.flux
                        for i in r0a.flux:
                            r0a.flux[i] = -1*r0a.flux[i]
                        #flip signs of r0a.wave
                        for i in r0a.wave:
                            r0a.wave[i] = -1*r0a.wave[i]
                    d0a = Spectra()
                    d0a.update_no_expid(s0a)#not working, produce NoneType
                    d0a.update_no_expid(r0a)#not working, produces NoneType
                    coadd(d0a)   #how to store differenced spectra as one per reference spectra will be produced
                    print("--- j:  %s seconds ---" % (time.time() - start_time))

In [78]:
s0 = Spectra()
s0 = read_spectra(f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/136/20210406/spectra-8-136-thru20210406.fits")

INFO:spectra.py:253:read_spectra: iotime 2.789 sec to read spectra-8-136-thru20210406.fits at 2021-07-14T20:59:52.318887


In [65]:
s0.fibermap['EXPID']

83539
83539
83539
83539
83539
83539
83539
83539
83539
83539
83539


In [67]:
s0.fibermap['NIGHT']

20210405
20210405
20210405
20210405
20210405
20210405
20210405
20210405
20210405
20210405
20210405


In [93]:
s0a = s0.select(nights=[20210406])

In [97]:
s0b.fibermap['EXPID']

83539
83539
83539
83539
83539
83539
83539
83539
83539
83539
83539


In [94]:
s0b = s0.select(nights=[20210405])

In [98]:
s0a.update(s0b)

In [99]:
s0a.fibermap['EXPID']

83723
83723
83723
83723
83723
83723
83723
83723
83723
83723
83723


In [79]:
s1 = Spectra()
s1 = read_spectra(f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/136/20210405/spectra-8-136-thru20210405.fits")

INFO:spectra.py:253:read_spectra: iotime 1.336 sec to read spectra-8-136-thru20210405.fits at 2021-07-14T21:00:13.129876


In [88]:
s1.fibermap['EXPID']

83539
83539
83539
83539
83539
83539
83539
83539
83539
83539
83539


In [75]:
s0.update(s1)

In [89]:
s0.fibermap['EXPID']

83539
83539
83539
83539
83539
83539
83539
83539
83539
83539
83539


In [76]:
coadd(s0)

In [77]:
s0.fibermap['EXPID']

KeyError: 'EXPID'